In [17]:
from numpy.core.fromnumeric import mean
from flask import jsonify
import pandas as pd
import numpy as np
import json
import re
import http
import xgboost as xgb
import time
import urllib
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
from itertools import chain, starmap
%load_ext jupyternotify

# years completed 2002-2020
years_arr = range(2001,2005)
def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException, http.client.IncompleteRead) as e: 
        print('Download error:', url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
        if num_retries > 0: 
            if e == http.client.IncompleteRead: 
                time.sleep(10)
                return download(url, num_retries - 1)
    return html

def flatten_json_iterative(dictionary, sep = '.', ind_start = 0):
    """Flattening a nested json file"""

    def unpack_one(parent_key, parent_value):
        """Unpack one level (only one) of nesting in json file"""
        # Unpacking one level        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                t1 = parent_key + sep + key
                yield t1, value
        elif isinstance(parent_value, list):
            i = ind_start 
            for value in parent_value:
                t2 = parent_key + sep +str(i) 
                i += 1
                yield t2, value
        else:
            yield parent_key, parent_value    

            
    # Continue iterating the unpack_one function until the terminating condition is satisfied
    while True:
        # Continue unpacking the json file until all values are atomic elements (aka neither a dictionary nor a list)
        dictionary = dict(chain.from_iterable(starmap(unpack_one, dictionary.items())))
        # Terminating condition: none of the values in the json file are a dictionary or a list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

def nhl_pbp(gameId):
        """nba_pbp()
        Pull the game by id
        Data from API endpoints:
        * nhl/playbyplay
        * nhl/summary
        """
        # play by play
        pbp_url = "http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/summary?event={}".format(gameId)
        pbp_resp = download(url=pbp_url)
        pbp_txt = {}
        if pbp_resp is not None:
            pbp_txt['teams'] = np.array([])
            pbp_txt['id'] = np.array([])
            pbp_txt['competitions'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['header'] = {}
            pbp_txt['pickcenter'] = np.array([])
            pbp_txt['broadcasts'] = np.array([])
            pbp_txt['videos'] = np.array([])
            pbp_txt['standings'] = np.array([])
            pbp_txt['boxscore'] = np.array([])
            pbp_txt['espnWP'] = np.array([])
            pbp_txt['gameInfo'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['winprobability'] = np.array([])
            pbp_txt['timeouts'] = {}
            pbp_d = json.loads(pbp_resp)
            pbp_txt = pbp_d
            pbp_txt['winprobability'] = np.array([])
            pbp_txt['gameId'] = str(gameId)
            # summary endpoint for pickcenter array
            
            summary = pbp_txt
            
            if 'pickcenter' in summary.keys():
                summary_txt = summary['pickcenter']
            else:
                summary_txt = {}
                summary_txt['pickcenter']={}
            # ESPN's win probability
            wp = "winprobability"
            if wp in summary:
                espnWP = summary["winprobability"]
            else:
                espnWP = np.array([])
            if 'news' in pbp_txt.keys():
                del pbp_txt['news']
            if 'shop' in pbp_txt.keys():
                del pbp_txt['shop']
            pbp_txt['gameInfo'] = pbp_txt['header']['competitions'][0]
            pbp_txt['season'] = pbp_txt['header']['season']['year']
            pbp_txt['pickcenter'] = summary_txt
            pbp_txt['espnWP'] = espnWP
            # Home and Away identification variables
            homeTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['id'])
            awayTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['id'])
            homeTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['name'])
            awayTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['name'])
            homeTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['location'])
            awayTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['location'])
            homeTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['abbreviation'])
            awayTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['abbreviation'])
            homeTeamNameAlt = re.sub("Stat(.+)", "St", str(homeTeamName))
            awayTeamNameAlt = re.sub("Stat(.+)", "St", str(awayTeamName))

            if len(pbp_txt['espnWP']) > 1:
                pbp_txt['espnWP'] = espnWP
            else:
                pbp_txt['espnWP'] = espnWP
            pbp_txt['plays_mod'] = []
            for play in pbp_txt['plays']:
                p = flatten_json_iterative(play)
                pbp_txt['plays_mod'].append(p)
            pbp_txt['plays'] = pd.json_normalize(pbp_txt,'plays_mod')
            if len(pbp_txt['plays'])>1:
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                # Spread definition
                pbp_txt['plays']["homeTeamSpread"] = 2.5
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                pbp_txt['plays'] = pd.DataFrame(pbp_txt['plays'])
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']['period.number'] = pbp_txt['plays']['period.number'].apply(lambda x: int(x))
                #----- Figuring out Timeouts ---------
                pbp_txt['timeouts'] = {}
                pbp_txt['timeouts'][homeTeamId] = {"1": [], "2": []}
                pbp_txt['timeouts'][awayTeamId] = {"1": [], "2": []}
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                #----- Time ---------------
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.displayValue'].str.split(pat=':')
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.mm'].apply(lambda m: ['0'] + m if len(m) < 2 else m)
                pbp_txt['plays'][['clock.minutes','clock.seconds']] = pbp_txt['plays']['clock.mm'].to_list()
                pbp_txt['plays']['half'] = np.where(pbp_txt['plays']['period.number'] <= 2, "1","2")
                pbp_txt['plays']['lag_half'] = pbp_txt['plays']['half'].shift(1)
                pbp_txt['plays']['lead_half'] = pbp_txt['plays']['half'].shift(-1)

                pbp_txt['plays']['game_play_number'] = np.arange(len(pbp_txt['plays']))+1
                pbp_txt['plays']['text'] = pbp_txt['plays']['text'].astype(str)
                pbp_txt['plays']['id'] = pbp_txt['plays']['id'].apply(lambda x: int(x))
                del pbp_txt['plays']['clock.mm']
            else:
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
            
            
            pbp_json = {
                "gameId": pbp_txt['gameId'],
                "plays" : pbp_txt['plays'].to_dict(orient='records'),
                "winprobability" : np.array(pbp_txt['winprobability']).tolist(),
                "boxscore" : pbp_txt['boxscore'],
                "header" : pbp_txt['header'],
                "homeTeamSpread" : np.array(pbp_txt['homeTeamSpread']).tolist(),
                "broadcasts" : np.array(pbp_txt['broadcasts']).tolist(),
                "videos" : np.array(pbp_txt['videos']).tolist(),
                "standings" : pbp_txt['standings'],
                "pickcenter" : np.array(pbp_txt['pickcenter']).tolist(),
                "espnWP" : np.array(pbp_txt['espnWP']).tolist(),
                "gameInfo" : np.array(pbp_txt['gameInfo']).tolist(),
                "season" : np.array(pbp_txt['season']).tolist()
            }
            return pbp_json, pbp_json['season']
        else:
            pass
        
    
schedule = pd.read_csv('nhl_games_info_2002_2021.csv', encoding='latin-1')
schedule = schedule.sort_values(by=['season', 'date'], ascending = False)

games = schedule[(schedule['season'].isin(years_arr))&(schedule['status.type.name']=='STATUS_FINAL')].reset_index()['id']
print(f"Number of Games: {len(games)}, first gameId: {games[0]}")
g, y = nhl_pbp(gameId = games[0])
# for key in g.keys():
#     print(f"{key}: {type(g[key])}")    
json.dumps(g,indent=4)
print(g.keys())
print(pd.DataFrame(g['plays']))
y

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


2021-02-28 23:16:38,927 [63340] WARNING  py.warnings:109: [JupyterRequire] C:\Users\saiem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (45,57,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Number of Games: 2749, first gameId: 230609011
dict_keys(['gameId', 'plays', 'winprobability', 'boxscore', 'header', 'homeTeamSpread', 'broadcasts', 'videos', 'standings', 'pickcenter', 'espnWP', 'gameInfo', 'season'])
                 id sequenceNumber type.id type.text type.abbreviation  \
0  2306090113204308              0     509   Penalty           penalty   
1  2306090113214310              0     509   Penalty           penalty   
2  2306090111210958              0     505      Goal              goal   
3  2306090113211254              0     509   Penalty           penalty   
4  2306090111212919              0     505      Goal              goal   
5  2306090111223208              0     505      Goal              goal   
6  2306090113223329              0     509   Penalty           penalty   

                                                text  awayScore  homeScore  \
0  Penalty to Turner Stevenson 2 minutes for Boar...          0          0   
1  Penalty to Turner Stevenson 2

2003

In [18]:
%%notify
# 2020:
# 2019: 
# 2018:
# 2017:
# 2016:
# 2015:
# 2014:
# 2013:
# 2012:
# 2011:
# 2010:
# 2009:
# 2008:
# 2007:
# 2006: 
# 2005:
# 2004:
# 2003:
# 2002:
i = 0
for game in games[i:]:
    print(f"Working on game {i+1} of {len(games)}, gameId: {games[i]}")
    if len(str(game))<9:
        i+=1
        continue
    try:
        g, y = nhl_pbp(gameId=game)
    except (TypeError) as e: 
        print("TypeError: yo", e)
        g, y = nhl_pbp(gameId=game)
        
    except (KeyError) as e:
        print("KeyError: yo", e)
        i+=1
        continue
    except (ValueError) as e:
        print("DecodeError: yo", e)
        i+=1
        continue
    fp = "nhl/{}/{}.json".format(y, game)
    with open(fp,'w') as f:
        json.dump(g, f, indent=4, sort_keys=False) 
#     time.sleep(1)        
    i+=1

Working on game 1 of 2749, gameId: 230609011
Working on game 2 of 2749, gameId: 230607025
Working on game 3 of 2749, gameId: 230605011
Working on game 4 of 2749, gameId: 230602025
Working on game 5 of 2749, gameId: 230531025
Working on game 6 of 2749, gameId: 230529011
Working on game 7 of 2749, gameId: 230527011
Working on game 8 of 2749, gameId: 230523014
Working on game 9 of 2749, gameId: 230521011
Working on game 10 of 2749, gameId: 230519014
Working on game 11 of 2749, gameId: 230517011
Working on game 12 of 2749, gameId: 230516025
Working on game 13 of 2749, gameId: 230515011
Working on game 14 of 2749, gameId: 230514025
Working on game 15 of 2749, gameId: 230513014
Working on game 16 of 2749, gameId: 230512030
Working on game 17 of 2749, gameId: 230510014
Working on game 18 of 2749, gameId: 230510030
Working on game 19 of 2749, gameId: 230508022
Working on game 20 of 2749, gameId: 230507030
Working on game 21 of 2749, gameId: 230505022
Working on game 22 of 2749, gameId: 2305050

Working on game 178 of 2749, gameId: 230325008
Working on game 179 of 2749, gameId: 230325022
Working on game 180 of 2749, gameId: 230325004
Working on game 181 of 2749, gameId: 230325027
Working on game 182 of 2749, gameId: 230325005
Working on game 183 of 2749, gameId: 230325010
Working on game 184 of 2749, gameId: 230325007
Working on game 185 of 2749, gameId: 230325014
Working on game 186 of 2749, gameId: 230324018
Working on game 187 of 2749, gameId: 230324025
Working on game 188 of 2749, gameId: 230324003
Working on game 189 of 2749, gameId: 230324026
Working on game 190 of 2749, gameId: 230324001
Working on game 191 of 2749, gameId: 230324002
Working on game 192 of 2749, gameId: 230324015
Working on game 193 of 2749, gameId: 230323006
Working on game 194 of 2749, gameId: 230323022
Working on game 195 of 2749, gameId: 230323030
Working on game 196 of 2749, gameId: 230323009
Working on game 197 of 2749, gameId: 230323004
Working on game 198 of 2749, gameId: 230322008
Working on ga

Working on game 353 of 2749, gameId: 230301010
Working on game 354 of 2749, gameId: 230301012
Working on game 355 of 2749, gameId: 230301013
Working on game 356 of 2749, gameId: 230301014
Working on game 357 of 2749, gameId: 230301021
Working on game 358 of 2749, gameId: 230301029
Working on game 359 of 2749, gameId: 230301008
Working on game 360 of 2749, gameId: 230301017
Working on game 361 of 2749, gameId: 230301027
Working on game 362 of 2749, gameId: 230301001
Working on game 363 of 2749, gameId: 230228024
Working on game 364 of 2749, gameId: 230228002
Working on game 365 of 2749, gameId: 230227022
Working on game 366 of 2749, gameId: 230227017
Working on game 367 of 2749, gameId: 230227013
Working on game 368 of 2749, gameId: 230227015
Working on game 369 of 2749, gameId: 230227019
Working on game 370 of 2749, gameId: 230227027
Working on game 371 of 2749, gameId: 230227005
Working on game 372 of 2749, gameId: 230227010
Working on game 373 of 2749, gameId: 230227012
Working on ga

Working on game 528 of 2749, gameId: 230205009
Working on game 529 of 2749, gameId: 230205030
Working on game 530 of 2749, gameId: 230205026
Working on game 531 of 2749, gameId: 230205013
Working on game 532 of 2749, gameId: 230205023
Working on game 533 of 2749, gameId: 230205029
Working on game 534 of 2749, gameId: 230204003
Working on game 535 of 2749, gameId: 230204005
Working on game 536 of 2749, gameId: 230204010
Working on game 537 of 2749, gameId: 230204011
Working on game 538 of 2749, gameId: 230204016
Working on game 539 of 2749, gameId: 230204001
Working on game 540 of 2749, gameId: 230204012
Working on game 541 of 2749, gameId: 230204020
Working on game 542 of 2749, gameId: 230202031
Working on game 543 of 2749, gameId: 230130008
Working on game 544 of 2749, gameId: 230130018
Working on game 545 of 2749, gameId: 230130022
Working on game 546 of 2749, gameId: 230130001
Working on game 547 of 2749, gameId: 230130013
Working on game 548 of 2749, gameId: 230130019
Working on ga

Working on game 703 of 2749, gameId: 230109020
Working on game 704 of 2749, gameId: 230108025
Working on game 705 of 2749, gameId: 230108022
Working on game 706 of 2749, gameId: 230108004
Working on game 707 of 2749, gameId: 230108030
Working on game 708 of 2749, gameId: 230108026
Working on game 709 of 2749, gameId: 230108013
Working on game 710 of 2749, gameId: 230107017
Working on game 711 of 2749, gameId: 230107009
Working on game 712 of 2749, gameId: 230107027
Working on game 713 of 2749, gameId: 230107011
Working on game 714 of 2749, gameId: 230107021
Working on game 715 of 2749, gameId: 230107028
Working on game 716 of 2749, gameId: 230107012
Working on game 717 of 2749, gameId: 230107015
Working on game 718 of 2749, gameId: 230107020
Working on game 719 of 2749, gameId: 230106018
Working on game 720 of 2749, gameId: 230106013
Working on game 721 of 2749, gameId: 230106030
Working on game 722 of 2749, gameId: 230106029
Working on game 723 of 2749, gameId: 230105004
Working on ga

...

Working on game 2078 of 2749, gameId: 220112001
Working on game 2079 of 2749, gameId: 220112002
Working on game 2080 of 2749, gameId: 220112021
Working on game 2081 of 2749, gameId: 220112029
Working on game 2082 of 2749, gameId: 220112018
Working on game 2083 of 2749, gameId: 220112005
Working on game 2084 of 2749, gameId: 220112015
Working on game 2085 of 2749, gameId: 220112016
Working on game 2086 of 2749, gameId: 220111030
Working on game 2087 of 2749, gameId: 220111026
Working on game 2088 of 2749, gameId: 220111028
Working on game 2089 of 2749, gameId: 220111023
Working on game 2090 of 2749, gameId: 220110006
Working on game 2091 of 2749, gameId: 220110004
Working on game 2092 of 2749, gameId: 220110027
Working on game 2093 of 2749, gameId: 220110010
Working on game 2094 of 2749, gameId: 220110015
Working on game 2095 of 2749, gameId: 220110001
Working on game 2096 of 2749, gameId: 220110002
Working on game 2097 of 2749, gameId: 220109025
Working on game 2098 of 2749, gameId: 22

Working on game 2249 of 2749, gameId: 211218012
Working on game 2250 of 2749, gameId: 211218015
Working on game 2251 of 2749, gameId: 211217024
Working on game 2252 of 2749, gameId: 211217009
Working on game 2253 of 2749, gameId: 211217005
Working on game 2254 of 2749, gameId: 211217010
Working on game 2255 of 2749, gameId: 211217013
Working on game 2256 of 2749, gameId: 211216025
Working on game 2257 of 2749, gameId: 211216015
Working on game 2258 of 2749, gameId: 211216030
Working on game 2259 of 2749, gameId: 211216016
Working on game 2260 of 2749, gameId: 211215008
Working on game 2261 of 2749, gameId: 211215022
Working on game 2262 of 2749, gameId: 211215024
Working on game 2263 of 2749, gameId: 211215019
Working on game 2264 of 2749, gameId: 211215027
Working on game 2265 of 2749, gameId: 211215001
Working on game 2266 of 2749, gameId: 211215012
Working on game 2267 of 2749, gameId: 211215013
Working on game 2268 of 2749, gameId: 211215014
Working on game 2269 of 2749, gameId: 21

Working on game 2420 of 2749, gameId: 211123029
Working on game 2421 of 2749, gameId: 211123030
Working on game 2422 of 2749, gameId: 211123001
Working on game 2423 of 2749, gameId: 211122006
Working on game 2424 of 2749, gameId: 211122014
Working on game 2425 of 2749, gameId: 211122028
Working on game 2426 of 2749, gameId: 211121018
Working on game 2427 of 2749, gameId: 211121009
Working on game 2428 of 2749, gameId: 211121027
Working on game 2429 of 2749, gameId: 211121002
Working on game 2430 of 2749, gameId: 211121016
Working on game 2431 of 2749, gameId: 211121026
Working on game 2432 of 2749, gameId: 211121012
Working on game 2433 of 2749, gameId: 211121023
Working on game 2434 of 2749, gameId: 211121029
Working on game 2435 of 2749, gameId: 211120003
Working on game 2436 of 2749, gameId: 211120006
Working on game 2437 of 2749, gameId: 211120024
Working on game 2438 of 2749, gameId: 211120013
Working on game 2439 of 2749, gameId: 211120005
Working on game 2440 of 2749, gameId: 21

Working on game 2591 of 2749, gameId: 211027018
Working on game 2592 of 2749, gameId: 211027006
Working on game 2593 of 2749, gameId: 211027003
Working on game 2594 of 2749, gameId: 211027027
Working on game 2595 of 2749, gameId: 211027011
Working on game 2596 of 2749, gameId: 211027001
Working on game 2597 of 2749, gameId: 211027010
Working on game 2598 of 2749, gameId: 211027014
Working on game 2599 of 2749, gameId: 211027021
Working on game 2600 of 2749, gameId: 211027028
Working on game 2601 of 2749, gameId: 211027024
Working on game 2602 of 2749, gameId: 211026005
Working on game 2603 of 2749, gameId: 211026026
Working on game 2604 of 2749, gameId: 211026028
Working on game 2605 of 2749, gameId: 211026002
Working on game 2606 of 2749, gameId: 211026007
Working on game 2607 of 2749, gameId: 211025017
Working on game 2608 of 2749, gameId: 211025003
Working on game 2609 of 2749, gameId: 211025004
Working on game 2610 of 2749, gameId: 211025019
Working on game 2611 of 2749, gameId: 21

<IPython.core.display.Javascript object>